In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.c

In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd
# Defining my email dataset
my_dataset = {
    "instruction": [
        "Please respond to the following email inquiry:",
        "Please provide assistance regarding the following customer inquiry:",
        "Respond to the customer's inquiry with relevant information:",
        "Reply to the following email with details about our subscription plans:",
        "Please address the login issue reported in the following email:",
    ],
    "output": [
        "Sure, here's an overview of our products and services...",
        "Of course, I'd be happy to assist you with your account issues...",
        "Certainly, let me check on the status of your order and get back to you...",
        "Certainly, here are the subscription plans we offer along with their benefits...",
        "I apologize for the inconvenience. Let's troubleshoot the login issue together...",
    ]
}

# Converting the dataset into a DataFrame
my_df = pd.DataFrame(my_dataset)
my_hf_dataset = Dataset.from_pandas(my_df)
my_dataset_dict = DatasetDict({"train": my_hf_dataset})
def chat_template(example):
    example["instruction"] = f"### Instruction:\n{example['instruction']}\n\n### Response:\n"
    return example

# Applying the chat template function to the dataset
my_dataset_dict = my_dataset_dict.map(chat_template)
print(my_dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 5
    })
})


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 5
    })
})


In [ ]:
updated_df = my_dataset_dict["train"].to_pandas()

# Save the DataFrame to a CSV file
updated_df.to_csv("updated_dataset.csv", index=False)

In [ ]:
# Model
base_model = "meta-llama/Llama-2-7b-chat-hf"
new_model = "llama-2-7b-finetunedbypranav"

# Dataset
dataset = load_dataset("pranav1001/email_dataset", split="train")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

In [ ]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)

# Cast the layernorm in fp32, make output embedding layer require grads, add the upcasting of the lmhead to fp32
model = prepare_model_for_kbit_training(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Set training arguments
training_arguments = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,
        per_device_train_batch_size=10,
        gradient_accumulation_steps=1,
        evaluation_strategy="steps",
        eval_steps=1000,
        logging_steps=1,
        optim="paged_adamw_8bit",
        learning_rate=2e-4,
        lr_scheduler_type="linear",
        warmup_steps=10,
        report_to="wandb",
        max_steps=2,
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="instruction",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
)
trainer.train()
# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


In [ ]:
from transformers import pipeline

# Define the input text for testing
input_text = "Can you provide me with information about your subscription plans?"

# Construct the prompt with instruction and input text
prompt = f"""### Instruction:
Given the input provided below, respond to it in the format of an email.

Input:
{input_text}

Response:
"""

# Run text generation pipeline with your fine-tuned model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s>[INST] {prompt} [/INST]")

# Extract the generated email response and print it
generated_response = result[0]['generated_text'][len(prompt):]
print(generated_response)

Response:
 [/INST]  Subject: Re: Subscription Plans

Dear [Name],

Thank you for reaching out to us! We're happy to provide you with information about our subscription plans.

We offer a variety of subscription plans to suit your needs and budget. Here are our current plans:

* Basic: $10/month - This plan gives you access to our full library of courses, including exclusive content and updates.
* Premium: $20/month - Our premium plan includes everything in the basic plan, plus additional features such as personalized learning recommendations and priority customer support.
* Pro: $50/month - Our pro plan offers the full range of features in the premium plan, as well as additional benefits such as access to exclusive events and discounts on additional courses.

We also offer a free trial for all of our plans, so you can try them out before committing to a subscription.

If you have any questions or would like to sign up for a plan, please let us know!

Best regards,
[Your Name]
